<a href="https://colab.research.google.com/github/mattocanas/Novozyme-Stability-Prediction/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("\n... IMPORTS STARTING ...\n")

# Pip installs for competition specific stuff
!pip install -q biopandas
!pip install -q biopython
!pip install tensorflow-addons
!pip install tensorflow-io
!pip install pandarallel
!pip install Levenshtein
!pip install openslide
!pip install kaggle


print("\n\tVERSION INFORMATION")
# Machine Learning and Data Science Imports
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import tensorflow_hub as tfhub; print(f"\t\t– TENSORFLOW HUB VERSION: {tfhub.__version__}");
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import tensorflow_io as tfio; print(f"\t\t– TENSORFLOW I/O VERSION: {tfio.__version__}");
import Bio as pyb; print(f"\t\t– BioPython VERSION: {pyb.__version__}");
import biopandas; from biopandas.pdb import PandasPdb; print(f"\t\t– BioPandas VERSION: {biopandas.__version__}"); pdb = PandasPdb();
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");
import sklearn; print(f"\t\t– SKLEARN VERSION: {sklearn.__version__}");
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from pandarallel import pandarallel; pandarallel.initialize();
from sklearn.model_selection import GroupKFold, StratifiedKFold
from scipy.spatial import cKDTree

# # RAPIDS
# import cudf, cupy, cuml
# from cuml.neighbors import NearestNeighbors
# from cuml.manifold import TSNE, UMAP
# from cuml import PCA

# Built In Imports
from collections import Counter
from datetime import datetime
from zipfile import ZipFile
from glob import glob
import Levenshtein
# import openslide
import warnings
import requests
import hashlib
import imageio
import IPython
import sklearn
import urllib
import zipfile
import pickle
import random
import shutil
import string
import json
import math
import time
import gzip
import ast
import sys
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import tifffile as tif
import seaborn as sns
from PIL import Image, ImageEnhance; Image.MAX_IMAGE_PIXELS = 5_000_000_000;
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
from matplotlib import animation, rc; rc('animation', html='jshtml')
import plotly
import PIL
import cv2

import plotly.io as pio
print(pio.renderers)

def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    
print("\n\n... IMPORTS COMPLETE ...\n")


... IMPORTS STARTING ...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 35.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandarallel: filename=pandarallel-1.6.4-py3-none-any.whl size=16678 sha256=410bbf0364a3ceacbecaa1aec21dc7685fae7d7bca9ace6cef3f8ecc60984a74
  Stored in directory: /root/.cache/pip/wheels/62/1e/e7/f9ee096e5cc02890a6934a5670ff6e45a3400f330605bd8210
Successfully built pandarallel
Looking in indexe

In [2]:
print(f"\n... ACCELERATOR SETUP STARTING ...\n")

# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  
except ValueError:
    TPU = None

if TPU:
    print(f"\n... RUNNING ON TPU - {TPU.master()}...")
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.experimental.TPUStrategy(TPU)
else:
    print(f"\n... RUNNING ON CPU/GPU ...")
    # Yield the default distribution strategy in Tensorflow
    #   --> Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()
    _gpus = tf.config.list_physical_devices('GPU')
    if len(_gpus)>0: tf.config.experimental.set_memory_growth([0], True)

# What Is a Replica?
#    --> A single Cloud TPU device consists of FOUR chips, each of which has TWO TPU cores. 
#    --> Therefore, for efficient utilization of Cloud TPU, a program should make use of each of the EIGHT (4x2) cores. 
#    --> Each replica is essentially a copy of the training graph that is run on each core and 
#        trains a mini-batch containing 1/8th of the overall batch size
N_REPLICAS = strategy.num_replicas_in_sync
    
print(f"... # OF REPLICAS: {N_REPLICAS} ...\n")

print(f"\n... ACCELERATOR SETUP COMPLTED ...\n")


... ACCELERATOR SETUP STARTING ...


... RUNNING ON TPU - grpc://10.6.170.2:8470...


... # OF REPLICAS: 8 ...


... ACCELERATOR SETUP COMPLTED ...



In [3]:
print(f"\n... XLA OPTIMIZATIONS STARTING ...\n")

print(f"\n... CONFIGURE JIT (JUST IN TIME) COMPILATION ...\n")
# enable XLA optmizations (10% speedup when using @tf.function calls)
tf.config.optimizer.set_jit(True)

print(f"\n... XLA OPTIMIZATIONS COMPLETED ...\n")


... XLA OPTIMIZATIONS STARTING ...


... CONFIGURE JIT (JUST IN TIME) COMPILATION ...


... XLA OPTIMIZATIONS COMPLETED ...



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive/novozymes-enzyme-stability-prediction 4"


 AF70_mutations.txt	    train.csv
'sample_submission 2.csv'  'train_updates_20220929 2.csv'
 sample_submission.csv	    train_updates_20220929.csv
'test 2.csv'		    wildtype_af70.fasta
 test.csv		   'wildtype_structure_prediction_af2 2.pdb'
'train 2.csv'		    wildtype_structure_prediction_af2.pdb


In [6]:
pdb_df =  PandasPdb().read_pdb("/content/drive/My Drive/novozymes-enzyme-stability-prediction 4/wildtype_structure_prediction_af2.pdb")

print(pdb_df.df.keys())
atom_df = pdb_df.df['ATOM']
hetatm_df = pdb_df.df['HETATM']
anisou_df = pdb_df.df['ANISOU']
others_df = pdb_df.df['OTHERS']

dict_keys(['ATOM', 'HETATM', 'ANISOU', 'OTHERS'])


In [7]:
print("\n... BASIC DATA SETUP STARTING ...\n")

print("\n\n... LOAD TRAIN DATAFRAME FROM CSV FILE ...\n")
train_df = pd.read_csv("/content/drive/My Drive/novozymes-enzyme-stability-prediction 4/train.csv")
display(train_df)

print("\n\n... LOAD TEST DATAFRAME FROM CSV FILE ...\n")
test_df = pd.read_csv("/content/drive/My Drive/novozymes-enzyme-stability-prediction 4/test.csv")
display(test_df)

print("\n\n... LOAD SAMPLE SUBMISSION DATAFRAME FROM CSV FILE ...\n")
ss_df = pd.read_csv("/content/drive/My Drive/novozymes-enzyme-stability-prediction 4/sample_submission.csv")
display(ss_df)

print("\n\n... LOAD ALPHAFOLD WILDTYPE STRUCTURE DATA FROM PDB FILE ...\n")
pdb_df = pdb.read_pdb("/content/drive/My Drive/novozymes-enzyme-stability-prediction 4/wildtype_structure_prediction_af2.pdb")

print("ATOM DATA...")
atom_df   = pdb_df.df['ATOM']
display(atom_df)

print("\nHETATM DATA...")
hetatm_df = pdb_df.df['HETATM']
display(hetatm_df)

print("\nANISOU DATA...")
anisou_df = pdb_df.df['ANISOU']
display(anisou_df)

print("\nOTHERS DATA...")
others_df = pdb_df.df['OTHERS']
display(others_df)




... BASIC DATA SETUP STARTING ...



... LOAD TRAIN DATAFRAME FROM CSV FILE ...



,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5
...,...,...,...,...,...
31385,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8
31386,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2
31387,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6
31388,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.7




... LOAD TEST DATAFRAME FROM CSV FILE ...



,seq_id,protein_sequence,pH,data_source
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes




... LOAD SAMPLE SUBMISSION DATAFRAME FROM CSV FILE ...



...

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx



ANISOU DATA...


,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,"U(1,1)","U(2,2)","U(3,3)","U(1,2)","U(1,3)","U(2,3)",blank_4,element_symbol,charge,line_idx



OTHERS DATA...


,record_name,entry,line_idx
0,TER,3318 LYS A 221,3317
1,END,,3318


* Atom serial number: This is a unique identifier assigned to each atom in the structure. It is a sequential number assigned to each atom in the order they appear in the PDB file.

* Atom name: This is the name of the atom, which can be either a standard element symbol or a standard element symbol followed by a number to distinguish different atoms of the same element in the same residue.

* Alternate location indicator: This field is used to indicate the presence of alternative positions for an atom within a residue. If an atom has multiple positions, each will be given a unique alternate location identifier.

* Residue name: This is the name of the residue to which the atom belongs. Residues are the building blocks of proteins or nucleic acids and are typically composed of one or more atoms.

* Chain identifier: This is a single letter code used to identify the chain in which the residue is located.

* Residue sequence number: This is a unique identifier for the residue, assigned based on its position within the chain.

* Code for insertion of residues: This field is used to specify the presence of inserted residues, which are residues that have been added to the structure after its initial formation.

* Orthogonal coordinates for X, Y, and Z in Angstroms: These are the 3D coordinates of the atom in space, given in Angstroms. X, Y, and Z define the position of the atom relative to a fixed point in space.

* Occupancy: This is a value that indicates the fraction of the time the atom is found at its defined position in the crystal structure.

* **Temperature factor (also known as B factor): This is a measure of the thermal motion of the atom, given in Angstroms^2. A high B factor indicates high thermal motion and a low B factor indicates low thermal motion.**
  * a high B factor indicates that an atom deviates more from its ideal conformation due to a change (increase) in temperature
  * this can alter the stability or activity of an enzyme

* Element symbol: This is the standard symbol for the chemical element to which the atom belongs, such as C for carbon or N for nitrogen.

* Charge on the atom: This field is used to indicate the charge on the atom, if any. In most cases, atoms in proteins or nucleic acids are neutral and have a charge of zero.*

In [8]:
print("\n\n... SAVING WILDTYPE AMINO ACID SEQUENCE...\n")
wildtype_aa = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"
print(wildtype_aa)

print("\n\n... DEFINE AMINO ACID SHORTFORM DICTIONARY MAPPING...\n")
aa_map = dict(Alanine=("Ala", "A"), Arginine=("Arg", "R"), Asparagine=("Asn", "N"), Aspartic_Acid=("Asp", "D"),
              Cysteine=("Cys", "C"), Glutamic_Acid=("Glu", "E"), Glutamine=("Gln", "Q"), Glycine=("Gly", "G"),
              Histidine=("His", "H"), Isoleucine=("Ile", "I"), Leucine=("Leu", "L"), Lysine=("Lys", "K"),
              Methionine=("Met", "M"), Phenylalanine=("Phe", "F"), Proline=("Pro", "P"), Serine=("Ser", "S"),
              Threonine=("Thr", "T"), Tryptophan=("Trp", "W"), Tyrosine=("Tyr", "Y"), Valine=("Val", "V"))
n_aa = len(aa_map)
aa_chars_ordered = sorted([v[1] for v in aa_map.values()])
aa_long2tri = {k:v[0] for k,v in aa_map.items()} # map full name to 3-letter code
aa_long2char = {k:v[1] for k,v in aa_map.items()} # map full name to single-letter code
aa_tri2long = {v:k for k,v in aa_long2tri.items()} # map 3-letter code to full name
aa_char2long = {v:k for k,v in aa_long2char.items()} # map 1 letter code to full name
aa_char2int = {_aa:i for i, _aa in enumerate(aa_chars_ordered)} # map single letter code to number
aa_int2char = {v:k for k,v in aa_char2int.items()} # map number to single letter code






... SAVING WILDTYPE AMINO ACID SEQUENCE...

VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK


... DEFINE AMINO ACID SHORTFORM DICTIONARY MAPPING...



In [9]:
# Get data source map
print("\n\n... DEFINE DATASOURCE DICTIONARY MAPPING...\n")

# map each uniqe data source in the training set to an integer. then do the reverse, mapping each integer to each unique data source in the training set
ds_str2int = {k:i for i,k in enumerate(train_df["data_source"].unique())}
ds_int2str = {v:k for k,v in ds_str2int.items()}




... DEFINE DATASOURCE DICTIONARY MAPPING...



In [10]:
for k,v in aa_map.items(): print(f"'{k}':\n\t3 LETTER ABBREVIATION --> '{v[0]}'\n\t1 LETTER ABBREVIATION --> '{v[1]}'\n")

'Alanine':
	3 LETTER ABBREVIATION --> 'Ala'
	1 LETTER ABBREVIATION --> 'A'

'Arginine':
	3 LETTER ABBREVIATION --> 'Arg'
	1 LETTER ABBREVIATION --> 'R'

'Asparagine':
	3 LETTER ABBREVIATION --> 'Asn'
	1 LETTER ABBREVIATION --> 'N'

'Aspartic_Acid':
	3 LETTER ABBREVIATION --> 'Asp'
	1 LETTER ABBREVIATION --> 'D'

'Cysteine':
	3 LETTER ABBREVIATION --> 'Cys'
	1 LETTER ABBREVIATION --> 'C'

'Glutamic_Acid':
	3 LETTER ABBREVIATION --> 'Glu'
	1 LETTER ABBREVIATION --> 'E'

'Glutamine':
	3 LETTER ABBREVIATION --> 'Gln'
	1 LETTER ABBREVIATION --> 'Q'

'Glycine':
	3 LETTER ABBREVIATION --> 'Gly'
	1 LETTER ABBREVIATION --> 'G'

'Histidine':
	3 LETTER ABBREVIATION --> 'His'
	1 LETTER ABBREVIATION --> 'H'

'Isoleucine':
	3 LETTER ABBREVIATION --> 'Ile'
	1 LETTER ABBREVIATION --> 'I'

'Leucine':
	3 LETTER ABBREVIATION --> 'Leu'
	1 LETTER ABBREVIATION --> 'L'

'Lysine':
	3 LETTER ABBREVIATION --> 'Lys'
	1 LETTER ABBREVIATION --> 'K'

'Methionine':
	3 LETTER ABBREVIATION --> 'Met'
	1 LETTER ABBREVIA

In [11]:
print("\n\n... FOR FUN ... HERE IS THE ENTIRE WILDTYPE WITH FULL AMINO ACID NAMES (8 AA PER LINE) ...\n")
for i, _c in enumerate(wildtype_aa): print(f"'{aa_char2long[_c]}'", end=", ") if (i+1)%8!=0 else print(f"{aa_char2long[_c]}", end=",\n")



... FOR FUN ... HERE IS THE ENTIRE WILDTYPE WITH FULL AMINO ACID NAMES (8 AA PER LINE) ...

'Valine', 'Proline', 'Valine', 'Asparagine', 'Proline', 'Glutamic_Acid', 'Proline', Aspartic_Acid,
'Alanine', 'Threonine', 'Serine', 'Valine', 'Glutamic_Acid', 'Asparagine', 'Valine', Alanine,
'Leucine', 'Lysine', 'Threonine', 'Glycine', 'Serine', 'Glycine', 'Aspartic_Acid', Serine,
'Glutamine', 'Serine', 'Aspartic_Acid', 'Proline', 'Isoleucine', 'Lysine', 'Alanine', Aspartic_Acid,
'Leucine', 'Glutamic_Acid', 'Valine', 'Lysine', 'Glycine', 'Glutamine', 'Serine', Alanine,
'Leucine', 'Proline', 'Phenylalanine', 'Aspartic_Acid', 'Valine', 'Aspartic_Acid', 'Cysteine', Tryptophan,
'Alanine', 'Isoleucine', 'Leucine', 'Cysteine', 'Lysine', 'Glycine', 'Alanine', Proline,
'Asparagine', 'Valine', 'Leucine', 'Glutamine', 'Arginine', 'Valine', 'Asparagine', Glutamic_Acid,
'Lysine', 'Threonine', 'Lysine', 'Asparagine', 'Serine', 'Asparagine', 'Arginine', Aspartic_Acid,
'Arginine', 'Serine', 'Glycine', 'Ala

In [12]:
print("\n\n... ADD COLUMNS FOR PROTEIN SEQUENCE LENGTH AND INDIVIDUAL AMINO ACID COUNTS/FRACTIONS ...\n")
train_df["n_AA"] = train_df["protein_sequence"].apply(len)
test_df["n_AA"] = test_df["protein_sequence"].apply(len)
for _aa_char in aa_chars_ordered: 
    train_df[f"AA_{_aa_char}__count"] = train_df["protein_sequence"].apply(lambda x: x.count(_aa_char))
    train_df[f"AA_{_aa_char}__fraction"] = train_df[f"AA_{_aa_char}__count"]/train_df["n_AA"]
    test_df[f"AA_{_aa_char}__count"] = test_df["protein_sequence"].apply(lambda x: x.count(_aa_char))
    test_df[f"AA_{_aa_char}__fraction"] = test_df[f"AA_{_aa_char}__count"]/test_df["n_AA"]



... ADD COLUMNS FOR PROTEIN SEQUENCE LENGTH AND INDIVIDUAL AMINO ACID COUNTS/FRACTIONS ...



In [13]:
print("\n... ADD COLUMNS FOR DATA SOURCE ENUMERATION ...\n")
train_df["data_source_enum"] = train_df['data_source'].map(ds_str2int)
test_df["data_source_enum"] = test_df['data_source'].map(ds_str2int)


... ADD COLUMNS FOR DATA SOURCE ENUMERATION ...



In [14]:
# if pH > 14, it is likely just an accidental swap between tm and pH value because there cannot be pH >14. so swap them around
print("\n... DO TEMPORARY pH FIX BY SWAPPING pH & tm IF pH>14 ...\n")
def tmp_ph_fix(_row):
    if _row["pH"]>14:
        print(f"\t--> pH WEIRDNESS EXISTS AT INDEX {_row.name}")
        _tmp = _row["pH"]
        _row["pH"] = _row["tm"]
        _row["tm"] = _tmp
        return _row
    else:
        return _row

print(f"\t--> DOES THE  STILL EXIST: {train_df['pH'].max()>14.0}")
train_df = train_df.progress_apply(tmp_ph_fix, axis=1)
test_df = test_df.progress_apply(tmp_ph_fix, axis=1)
    
print("\n\n\n... BASIC DATA SETUP FINISHED ...\n\n")


... DO TEMPORARY pH FIX BY SWAPPING pH & tm IF pH>14 ...

	--> DOES THE  STILL EXIST: True


  0%|          | 0/31390 [00:00<?, ?it/s]

	--> pH WEIRDNESS EXISTS AT INDEX 973
	--> pH WEIRDNESS EXISTS AT INDEX 986
	--> pH WEIRDNESS EXISTS AT INDEX 988
	--> pH WEIRDNESS EXISTS AT INDEX 989
	--> pH WEIRDNESS EXISTS AT INDEX 1003
	--> pH WEIRDNESS EXISTS AT INDEX 1012
	--> pH WEIRDNESS EXISTS AT INDEX 1014
	--> pH WEIRDNESS EXISTS AT INDEX 1018
	--> pH WEIRDNESS EXISTS AT INDEX 1037
	--> pH WEIRDNESS EXISTS AT INDEX 1042
	--> pH WEIRDNESS EXISTS AT INDEX 1079
	--> pH WEIRDNESS EXISTS AT INDEX 1081
	--> pH WEIRDNESS EXISTS AT INDEX 1088
	--> pH WEIRDNESS EXISTS AT INDEX 1093
	--> pH WEIRDNESS EXISTS AT INDEX 1096
	--> pH WEIRDNESS EXISTS AT INDEX 1098
	--> pH WEIRDNESS EXISTS AT INDEX 1100
	--> pH WEIRDNESS EXISTS AT INDEX 1108
	--> pH WEIRDNESS EXISTS AT INDEX 1111
	--> pH WEIRDNESS EXISTS AT INDEX 1120
	--> pH WEIRDNESS EXISTS AT INDEX 1122
	--> pH WEIRDNESS EXISTS AT INDEX 1125
	--> pH WEIRDNESS EXISTS AT INDEX 13447
	--> pH WEIRDNESS EXISTS AT INDEX 13449
	--> pH WEIRDNESS EXISTS AT INDEX 14640
	--> pH WEIRDNESS EXISTS A

  0%|          | 0/2413 [00:00<?, ?it/s]




... BASIC DATA SETUP FINISHED ...




## Helper Functions and Classes Below

Sets the wildtype sequence and takes a dictionary that contains an amino acid sequence.

Sets the terminology map. Levenshtein returns "replace", "insert", and "delete". In biological mutation terms, there are: "substitution", "insertion", "deletion".

Use Levenshetin.editops() to find all the operations needed to change the input sequence to the wild tpye. Accoding to chatgpt, this is how levenshetin works:


```
s1 = "apple"
s2 = "banana"
editops = Levenshtein.editops(s1, s2)
editops
----
[('replace', 0, 0), ('insert', 3, 4), ('replace', 4, 5), ('replace', 5, 6)]
```
This output shows that the first operation to turn s2 to s1 is to replace the 0th index. then make an insertion at the 3rd index of s2. The value to be inserted there can be found at the 4th index of s1.


If no len(req_edits) == 0. then there are no edits to be made. essentially, the input sequence is the same as the wildtype. so all the value are set to pd.NA

If there are edits to be made:
* find what kind of edit (the terminology)
* find the index that edit should occur at 
* find what the wild type amino acid is at that index
* find what the mutant amino acid is. if it is a deletion. set this to pd.NA



In [15]:
def get_mutation_info(_row, _wildtype="VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQ" \
                                      "RVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGT" \
                                      "NAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKAL" \
                                      "GSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"):
    terminology_map = {"replace":"substitution", "insert":"insertion", "delete":"deletion"}
    req_edits = Levenshtein.editops(_wildtype, _row["protein_sequence"])
    _row["n_edits"] = len(req_edits)
    
    if _row["n_edits"]==0:
        _row["edit_type"] = _row["edit_idx"] = _row["wildtype_aa"] = _row["mutant_aa"] = pd.NA
    else:
        _row["edit_type"] = terminology_map[req_edits[0][0]]
        _row["edit_idx"] = req_edits[0][1]
        _row["wildtype_aa"] = _wildtype[_row["edit_idx"]]
        _row["mutant_aa"] = _row["protein_sequence"][req_edits[0][2]] if _row["edit_type"]!="deletion" else pd.NA
    return _row



Checks to see if there is actually a difference between input and wildtype.. If not, the sequence is returned unaltered.

If the type of edit is not an insertion (replacement or deletion):
* creat a base that is the protein sequence up until the index that is edited.
* if it is a deletion, take the new base, add the wildtype amino acid, add the rest of the sequence until the end
* if it is a replacement, make the new wildtype equal to the base plus the wildtype AA plus the rest of the protein sequence, but skipping the AA at the replacement point
* Finally, if the edit type is an insertion. set the new wild_type equal to the protein sequence up until the edit, plus the wild type AA, plus the rest of the sequence

In [16]:
def revert_to_wildtype(protein_sequence, edit_type, edit_idx, wildtype_aa, mutant_aa):
    if pd.isna(edit_type):
        return protein_sequence
    elif edit_type!="insertion":
        new_wildtype_base = protein_sequence[:edit_idx]
        if edit_type=="deletion":
            new_wildtype=new_wildtype_base+wildtype_aa+protein_sequence[edit_idx:]
        else:
            new_wildtype=new_wildtype_base+wildtype_aa+protein_sequence[edit_idx+1:]
    else:
        new_wildtype=protein_sequence[:edit_idx]+wildtype_aa+protein_sequence[edit_idx:]
    return new_wildtype

The code below turns a list of lists into one big list.

Then there is a function that just prints a line of 110 characters

In [17]:
def flatten_l_o_l(nested_list):
    """ Flatten a list of lists """
    return [item for sublist in nested_list for item in sublist]

def print_ln(symbol="-", line_len=110):
    print(symbol*line_len)

## Training Set Exploration

In [18]:
# Get number of training examples
n_train = len(train_df)

print("\n... DATAFRAME INFO ...\n")
train_df.info()

print("\n\n\n... RELEVANT DATAFRAME COLUMN DESCRIPTIONS ...\n")
display(train_df.describe().T)


... DATAFRAME INFO ...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31390 entries, 0 to 31389
Data columns (total 47 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   seq_id            31390 non-null  int64  
 1   protein_sequence  31390 non-null  object 
 2   pH                31104 non-null  float64
 3   data_source       28043 non-null  object 
 4   tm                31390 non-null  float64
 5   n_AA              31390 non-null  int64  
 6   AA_A__count       31390 non-null  int64  
 7   AA_A__fraction    31390 non-null  float64
 8   AA_C__count       31390 non-null  int64  
 9   AA_C__fraction    31390 non-null  float64
 10  AA_D__count       31390 non-null  int64  
 11  AA_D__fraction    31390 non-null  float64
 12  AA_E__count       31390 non-null  int64  
 13  AA_E__fraction    31390 non-null  float64
 14  AA_F__count       31390 non-null  int64  
 15  AA_F__fraction    31390 non-null  float64
 16  AA_G__count    

,count,mean,std,min,25%,50%,75%,max
seq_id,31390.0,15694.500000,9061.656811,0.0,7847.250000,15694.500000,23541.750000,31389.000000
pH,31104.0,6.849342,0.865250,0.0,7.000000,7.000000,7.000000,11.000000
tm,31390.0,49.189943,13.946754,-1.0,42.100000,48.000000,53.800000,130.000000
n_AA,31390.0,447.669513,640.728935,5.0,197.000000,336.000000,523.000000,32767.000000
AA_A__count,31390.0,34.297579,43.774015,0.0,15.000000,26.000000,43.000000,1989.000000
AA_A__fraction,31390.0,0.079204,0.028294,0.0,0.059281,0.076479,0.094862,0.298658
AA_C__count,31390.0,6.605607,12.763594,0.0,2.000000,4.000000,8.000000,498.000000
AA_C__fraction,31390.0,0.014694,0.013378,0.0,0.006369,0.012195,0.019355,0.160714
AA_D__count,31390.0,24.880280,35.269664,0.0,10.000000,18.000000,30.000000,1660.000000
AA_D__fraction,31390.0,0.055320,0.017783,0.0,0.043780,0.054299,0.064877,0.205298


### Looking into the "Seq_Id" column

Observations:
* each sequence does indeed have a unique id 👍

In [19]:
print("\n\n... CHECK THAT THE NUMBER OF UNIQUE `seq_id` VALUES IS EQUAL TO THE NUMBER OF EXAMPLES `n_train` ...")
print(f"\t`train_df['seq_id'].nunique()==n_train` --> {train_df['seq_id'].nunique()==n_train}\n")



... CHECK THAT THE NUMBER OF UNIQUE `seq_id` VALUES IS EQUAL TO THE NUMBER OF EXAMPLES `n_train` ...
	`train_df['seq_id'].nunique()==n_train` --> True



### Looking into the "Protein_Sequence" Column

* check the correlation between each amino acid's frequency, and the tm score of that sequence
  * this is done for both the count and the fractional frequency

Observations:
* The frequency of some amino acids respective to the length of the AA sequence has a low, but non-zero correlation with the target *tm* (melting point)
* The fractional frequency appears slightly more informative*

In [20]:
for _aa_char in aa_chars_ordered:    
    # Check Correlations
    print(f"AA_{_aa_char} has a spearman correlation of : (" \
          f"{np.abs(train_df[f'AA_{_aa_char}__count'].corr(train_df['tm'], 'spearman')):.3f}, " \
          f"{np.abs(train_df[f'AA_{_aa_char}__fraction'].corr(train_df['tm'], 'spearman')):.3f}) (count, fraction)")

AA_A has a spearman correlation of : (0.006, 0.094) (count, fraction)
AA_C has a spearman correlation of : (0.123, 0.098) (count, fraction)
AA_D has a spearman correlation of : (0.089, 0.084) (count, fraction)
AA_E has a spearman correlation of : (0.029, 0.047) (count, fraction)
AA_F has a spearman correlation of : (0.073, 0.045) (count, fraction)
AA_G has a spearman correlation of : (0.011, 0.117) (count, fraction)
AA_H has a spearman correlation of : (0.058, 0.028) (count, fraction)
AA_I has a spearman correlation of : (0.085, 0.048) (count, fraction)
AA_K has a spearman correlation of : (0.156, 0.175) (count, fraction)
AA_L has a spearman correlation of : (0.004, 0.101) (count, fraction)
AA_M has a spearman correlation of : (0.097, 0.089) (count, fraction)
AA_N has a spearman correlation of : (0.132, 0.133) (count, fraction)
AA_P has a spearman correlation of : (0.007, 0.090) (count, fraction)
AA_Q has a spearman correlation of : (0.114, 0.109) (count, fraction)
AA_R has a spearman 

* check to see if identical AA sequences have identical tm values

Observations: 
* they do not.
* for some reason or another, identical protein sequences do not have identical tm scores. i guess this may be do to different groups measuring the tm of them. different conditions, wtc

In [21]:
# this code enumerates over duplicate protein sequences in the training set. it groups the identical ones together, and stops after a max of 5 identical sequences. it then displays the groups
for i, protein_group in enumerate(train_df[train_df["protein_sequence"].duplicated(keep=False)].groupby("protein_sequence")):
    if i==5: break
    display(protein_group[1])

,seq_id,protein_sequence,pH,data_source,tm,n_AA,AA_A__count,AA_A__fraction,AA_C__count,AA_C__fraction,...,AA_S__fraction,AA_T__count,AA_T__fraction,AA_V__count,AA_V__fraction,AA_W__count,AA_W__fraction,AA_Y__count,AA_Y__fraction,data_source_enum
972,972,DTSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFQNRESVLPT...,7.0,10.1074/jbc.M304177200,45.2,96,6,0.0625,2,0.020833,...,0.072917,14,0.145833,5,0.052083,0,0.0,8,0.083333,5
973,973,DTSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFQNRESVLPT...,7.0,NaN,48.4,96,6,0.0625,2,0.020833,...,0.072917,14,0.145833,5,0.052083,0,0.0,8,0.083333,1
974,974,DTSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFQNRESVLPT...,7.0,NaN,48.0,96,6,0.0625,2,0.020833,...,0.072917,14,0.145833,5,0.052083,0,0.0,8,0.083333,1


,seq_id,protein_sequence,pH,data_source,tm,n_AA,AA_A__count,AA_A__fraction,AA_C__count,AA_C__fraction,...,AA_S__fraction,AA_T__count,AA_T__fraction,AA_V__count,AA_V__fraction,AA_W__count,AA_W__fraction,AA_Y__count,AA_Y__fraction,data_source_enum
982,982,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQAGVVFQNRESVLPT...,7.0,10.1016/j.jmb.2005.09.091,31.8,96,7,0.072917,2,0.020833,...,0.072917,13,0.135417,6,0.0625,0,0.0,8,0.083333,6
983,983,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQAGVVFQNRESVLPT...,7.0,NaN,52.6,96,7,0.072917,2,0.020833,...,0.072917,13,0.135417,6,0.0625,0,0.0,8,0.083333,1
984,984,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQAGVVFQNRESVLPT...,7.0,NaN,25.0,96,7,0.072917,2,0.020833,...,0.072917,13,0.135417,6,0.0625,0,0.0,8,0.083333,1


,seq_id,protein_sequence,pH,data_source,tm,n_AA,AA_A__count,AA_A__fraction,AA_C__count,AA_C__fraction,...,AA_S__fraction,AA_T__count,AA_T__fraction,AA_V__count,AA_V__fraction,AA_W__count,AA_W__fraction,AA_Y__count,AA_Y__fraction,data_source_enum
985,985,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVTFQNRESVLPT...,7.0,10.1074/jbc.M304177200,43.8,96,6,0.0625,2,0.020833,...,0.072917,14,0.145833,5,0.052083,0,0.0,8,0.083333,5
986,986,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVTFQNRESVLPT...,7.0,NaN,48.4,96,6,0.0625,2,0.020833,...,0.072917,14,0.145833,5,0.052083,0,0.0,8,0.083333,1
987,987,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVTFQNRESVLPT...,7.0,NaN,48.0,96,6,0.0625,2,0.020833,...,0.072917,14,0.145833,5,0.052083,0,0.0,8,0.083333,1


,seq_id,protein_sequence,pH,data_source,tm,n_AA,AA_A__count,AA_A__fraction,AA_C__count,AA_C__fraction,...,AA_S__fraction,AA_T__count,AA_T__fraction,AA_V__count,AA_V__fraction,AA_W__count,AA_W__fraction,AA_Y__count,AA_Y__fraction,data_source_enum
988,988,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFANRESVLPT...,7.0,NaN,49.0,96,7,0.072917,2,0.020833,...,0.072917,13,0.135417,6,0.0625,0,0.0,8,0.083333,1
989,989,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFANRESVLPT...,5.5,NaN,55.6,96,7,0.072917,2,0.020833,...,0.072917,13,0.135417,6,0.0625,0,0.0,8,0.083333,1
990,990,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFANRESVLPT...,6.2,NaN,52.3,96,7,0.072917,2,0.020833,...,0.072917,13,0.135417,6,0.0625,0,0.0,8,0.083333,1
991,991,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFANRESVLPT...,5.5,10.1110/ps.03176803,56.9,96,7,0.072917,2,0.020833,...,0.072917,13,0.135417,6,0.0625,0,0.0,8,0.083333,7
992,992,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFANRESVLPT...,6.3,NaN,51.9,96,7,0.072917,2,0.020833,...,0.072917,13,0.135417,6,0.0625,0,0.0,8,0.083333,1
993,993,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFANRESVLPT...,7.0,10.1110/ps.03176803,52.5,96,7,0.072917,2,0.020833,...,0.072917,13,0.135417,6,0.0625,0,0.0,8,0.083333,7
994,994,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFANRESVLPT...,7.0,NaN,49.0,96,7,0.072917,2,0.020833,...,0.072917,13,0.135417,6,0.0625,0,0.0,8,0.083333,1


,seq_id,protein_sequence,pH,data_source,tm,n_AA,AA_A__count,AA_A__fraction,AA_C__count,AA_C__fraction,...,AA_S__fraction,AA_T__count,AA_T__fraction,AA_V__count,AA_V__fraction,AA_W__count,AA_W__fraction,AA_Y__count,AA_Y__fraction,data_source_enum
995,995,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFQARESVLPT...,7.0,NaN,48.4,96,7,0.072917,2,0.020833,...,0.072917,13,0.135417,6,0.0625,0,0.0,8,0.083333,1
996,996,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFQARESVLPT...,7.0,NaN,48.0,96,7,0.072917,2,0.020833,...,0.072917,13,0.135417,6,0.0625,0,0.0,8,0.083333,1
997,997,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFQARESVLPT...,7.0,10.1021/bi9815243,40.8,96,7,0.072917,2,0.020833,...,0.072917,13,0.135417,6,0.0625,0,0.0,8,0.083333,8


Here I am plotting the distribution of amino acid sequence length.

Observations: 
* clearly most sequences are on the 100 order of magnitude. But some are realllllyyy long. Like the 7 that are >30,000 AA long 

In [22]:
print("\n\n... CHECK DISTRIBUTION OF AMINO ACID SEQUENCE LENGTHS WITHIN TRAINING DATA ...")
fig = px.histogram(train_df, "n_AA", log_y=True, title="<b>Distribution of Amino Acid Sequence Lengths <sub>Count (y-axis) is Log Scale</sub></b>")
fig.show()



... CHECK DISTRIBUTION OF AMINO ACID SEQUENCE LENGTHS WITHIN TRAINING DATA ...


Checkin for whole sequence repeats:

In the output, the first column represents the number of times a particular number of protein sequence repeats occur in the training data. The second column is the count of how many times that number of repeats occurs, and the third column is the frequency of that count.

For example, the first row in the output:
0 1 27675
means that there are 27,675 protein sequences in the training data that occur only once.

In [23]:
print("\n\n... CHECK DISTRIBUTION OF REPEATED PROTEIN SEQUENCES WITHIN TRAINING DATA ...")
display(pd.Series(train_df["protein_sequence"].value_counts().values).value_counts().to_frame(name="frequency").reset_index().rename(columns={"index":"n_seq_repeats"}).sort_values(by="n_seq_repeats").reset_index(drop=True))



... CHECK DISTRIBUTION OF REPEATED PROTEIN SEQUENCES WITHIN TRAINING DATA ...


,n_seq_repeats,frequency
0,1,27675
1,2,720
2,3,334
3,4,130
4,5,53
5,6,34
6,7,19
7,8,6
8,9,4
9,10,4


In [24]:
train_df

,seq_id,protein_sequence,pH,data_source,tm,n_AA,AA_A__count,AA_A__fraction,AA_C__count,AA_C__fraction,...,AA_S__fraction,AA_T__count,AA_T__fraction,AA_V__count,AA_V__fraction,AA_W__count,AA_W__fraction,AA_Y__count,AA_Y__fraction,data_source_enum
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7,341,45,0.131965,1,0.002933,...,0.032258,14,0.041056,37,0.108504,4,0.011730,3,0.008798,0
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5,286,28,0.097902,0,0.000000,...,0.048951,12,0.041958,13,0.045455,3,0.010490,3,0.010490,0
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5,497,50,0.100604,9,0.018109,...,0.066398,30,0.060362,30,0.060362,3,0.006036,16,0.032193,0
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2,265,20,0.075472,5,0.018868,...,0.060377,19,0.071698,14,0.052830,3,0.011321,4,0.015094,0
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5,1451,86,0.059269,14,0.009649,...,0.101999,120,0.082702,124,0.085458,16,0.011027,47,0.032391,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31385,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8,549,33,0.060109,12,0.021858,...,0.060109,18,0.032787,42,0.076503,13,0.023679,18,0.032787,0
31386,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2,469,37,0.078891,5,0.010661,...,0.078891,26,0.055437,34,0.072495,5,0.010661,14,0.029851,0
31387,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6,128,13,0.101562,1,0.007812,...,0.078125,6,0.046875,7,0.054688,4,0.031250,4,0.031250,0
31388,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.7,593,47,0.079258,5,0.008432,...,0.086003,32,0.053963,48,0.080944,3,0.005059,18,0.030354,0


### Exploring the 'pH' column

Display the frequency of each pH value.

Plot the distribution of pH values across the dataset

Observation:
* all training pH values fall between 0-11

In [25]:
print("\n\n\n... CHECK DISTRIBUTION OF pH VALUES WITHIN TRAINING DATA ...")
print("\nValue counts rounded to nearest whole value\n")
display(train_df["pH"].apply(lambda x: np.round(x*2)//2).value_counts(dropna=False).to_frame(name="frequency").reset_index().rename(columns={"index":"pH"}).sort_values(by="pH").reset_index(drop=True))
print("\nHistogram")
fig = px.histogram(train_df, "pH", nbins=200, log_y=True, title="<b>Distribution of pH Values <sub>Count (y-axis) is Log Scale</sub></b>")
fig.show()




... CHECK DISTRIBUTION OF pH VALUES WITHIN TRAINING DATA ...

Value counts rounded to nearest whole value



,pH,frequency
0,0.0,6
1,1.0,1
2,2.0,452
3,3.0,482
4,4.0,243
5,5.0,781
6,6.0,622
7,7.0,27537
8,8.0,778
9,9.0,156



Histogram


### Exploring the 'tm' column

* Since only the Spearman Correlation Coefficient will be used for the evaluation, the correct prediction of the relative order is more important than the absolute tm values.
* Higher tm means the protein variant is more stable.

Range: 
* -1 - 130
* Float, not int

In [26]:
print("\n\n\n... CHECK DISTRIBUTION OF tm WITHIN TRAINING DATA ...")
fig = px.histogram(train_df, "tm", nbins=200, title="<b>Distribution of tm Values</b>")
fig.show()




... CHECK DISTRIBUTION OF tm WITHIN TRAINING DATA ...


In [27]:
print("\n... TRAINING DATASET `tm` DISTRIBUTION INFORMATION... \n")
train_df["tm"].describe()


... TRAINING DATASET `tm` DISTRIBUTION INFORMATION... 



count    31390.000000
mean        49.189943
std         13.946754
min         -1.000000
25%         42.100000
50%         48.000000
75%         53.800000
max        130.000000
Name: tm, dtype: float64

## Exploring the Test Set

In [28]:
test_df = test_df.progress_apply(get_mutation_info, axis=1)


  0%|          | 0/2413 [00:00<?, ?it/s]

In [29]:
print("\n... COLUMNS ADDED TO THE TEST DATAFRAME ...\n")
display(test_df[["protein_sequence", "edit_type", "edit_idx", "wildtype_aa", "mutant_aa"]].head(10))


... COLUMNS ADDED TO THE TEST DATAFRAME ...



,protein_sequence,edit_type,edit_idx,wildtype_aa,mutant_aa
0,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,substitution,16,L,E
1,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,substitution,16,L,K
2,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,deletion,16,L,<NA>
3,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,substitution,17,K,C
4,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,substitution,17,K,F
5,VPVNPEPDATSVENVALGTGSGDSQSDPIKADLEVKGQSALPFDVD...,substitution,17,K,G
6,VPVNPEPDATSVENVALHTGSGDSQSDPIKADLEVKGQSALPFDVD...,substitution,17,K,H
7,VPVNPEPDATSVENVALITGSGDSQSDPIKADLEVKGQSALPFDVD...,substitution,17,K,I
8,VPVNPEPDATSVENVALKAGSGDSQSDPIKADLEVKGQSALPFDVD...,substitution,18,T,A
9,VPVNPEPDATSVENVALKCGSGDSQSDPIKADLEVKGQSALPFDVD...,substitution,18,T,C


In [30]:
test_df.fillna({'edit_idx': 0, 'edit_type': 'none'}, inplace=True)


In [31]:
fig = px.histogram(test_df, y="edit_idx", color="edit_type", nbins=221, orientation="h", height=2500, width=1200, text_auto=True, title="<b>Distribution of Edits Along Wildtype Amino Acid Sequence</b>")
fig.update_layout(showlegend=False, bargap=0.05)
fig.show()

Creates a .txt file with all the mutations from the test set. Has the wildtype AA, the position, and the new AA in each line

Create a fasta file with the wiltype AS sequence

In [32]:
# Note mutation edit_idx is offset by 1 as many tools require it to be 1 indexed to 0 indexed.
def create_mutation_txt_file(_test_df, filename="/content/drive/My Drive/novozymes-enzyme-stability-prediction 4/AF70_mutations.txt", return_mutation_list=False, include_deletions=False):
    if return_mutation_list: mutation_list = []        
    with open(filename, 'w') as f:
        for _, _row in _test_df[["protein_sequence", "edit_type", "edit_idx", "wildtype_aa", "mutant_aa"]].iterrows():
            if not include_deletions and (pd.isna(_row["edit_type"]) or _row["edit_type"]=="deletion"): continue
            f.write(f'{_row["wildtype_aa"]+str(_row["edit_idx"]+1)+(_row["mutant_aa"] if not pd.isna(_row["mutant_aa"]) else "")}\n')
            if return_mutation_list: mutation_list.append(f'{_row["wildtype_aa"]+str(_row["edit_idx"]+1)+(_row["mutant_aa"] if not pd.isna(_row["mutant_aa"]) else "")}')
    if return_mutation_list: return mutation_list
        
def create_wildtype_fasta_file(wildtype_sequence, filename="/content/drive/My Drive/novozymes-enzyme-stability-prediction 4/wildtype_af70.fasta"):
    with open(filename, 'w') as f: f.write(f">af70_wildtype\n{wildtype_sequence}")
        
create_mutation_txt_file(test_df)
create_wildtype_fasta_file(wildtype_aa)

## Exploring the Wildtype_Structure_Prediction_AF2.PDB


In [33]:
print("ATOM DATA...")
atom_df   = pdb_df.df['ATOM']

display(atom_df)


ATOM DATA...


,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,1,,N,,VAL,,A,1,,...,34.064,-6.456,50.464,1.0,45.11,,,N,NaN,0
1,ATOM,2,,H,,VAL,,A,1,,...,33.576,-6.009,51.228,1.0,45.11,,,H,NaN,1
2,ATOM,3,,H2,,VAL,,A,1,,...,33.882,-7.449,50.477,1.0,45.11,,,H,NaN,2
3,ATOM,4,,H3,,VAL,,A,1,,...,35.060,-6.323,50.566,1.0,45.11,,,H,NaN,3
4,ATOM,5,,CA,,VAL,,A,1,,...,33.643,-5.877,49.162,1.0,45.11,,,C,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3312,ATOM,3313,,NZ,,LYS,,A,221,,...,4.616,13.323,-4.301,1.0,93.80,,,N,NaN,3312
3313,ATOM,3314,,HZ1,,LYS,,A,221,,...,5.270,12.565,-4.432,1.0,93.80,,,H,NaN,3313
3314,ATOM,3315,,HZ2,,LYS,,A,221,,...,4.585,13.517,-3.310,1.0,93.80,,,H,NaN,3314
3315,ATOM,3316,,HZ3,,LYS,,A,221,,...,4.965,14.143,-4.776,1.0,93.80,,,H,NaN,3315


## Exploring Sample Submission

In [34]:
ss_df

,seq_id,tm
0,31390,0
1,31391,1
2,31392,2
3,31393,3
4,31394,4
...,...,...
2408,33798,2408
2409,33799,2409
2410,33800,2410
2411,33801,2411


# Baseline Model

In [35]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor

# find all the train data that when rounded equals 8. drop the rest from the dataset
train_8ph_df = train_df[train_df["pH"].apply(lambda x: np.round(x*2)//2)==8.0].reset_index(drop=True)
display(train_8ph_df)

# drop any remaining data where the sequence length exceeds 700 AA
train_8ph_df = train_8ph_df[train_8ph_df.n_AA<700]
display(train_8ph_df)



,seq_id,protein_sequence,pH,data_source,tm,n_AA,AA_A__count,AA_A__fraction,AA_C__count,AA_C__fraction,...,AA_S__fraction,AA_T__count,AA_T__fraction,AA_V__count,AA_V__fraction,AA_W__count,AA_W__fraction,AA_Y__count,AA_Y__fraction,data_source_enum
0,1021,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFQNRESVLPT...,8.5,10.1016/j.jmb.2005.09.091,50.4,96,7,0.072917,2,0.020833,...,0.072917,13,0.135417,6,0.062500,0,0.000000,8,0.083333,6
1,1031,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFQNRESVLPT...,8.5,10.1016/j.jmb.2005.09.091,42.3,96,6,0.062500,2,0.020833,...,0.072917,13,0.135417,6,0.062500,0,0.000000,8,0.083333,6
2,1052,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFQNRESVLPT...,8.5,10.1016/j.jmb.2005.09.091,51.8,96,6,0.062500,2,0.020833,...,0.072917,13,0.135417,6,0.062500,0,0.000000,8,0.083333,6
3,1065,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFQNRESVLPT...,8.5,10.1016/j.jmb.2005.09.091,46.3,96,6,0.062500,2,0.020833,...,0.072917,13,0.135417,6,0.062500,0,0.000000,8,0.083333,6
4,3318,MAARLLRVASAALGDTAGRWRLLARPRAGAGGLRGSRGPGLGGGAV...,8.5,NaN,25.0,186,20,0.107527,5,0.026882,...,0.075269,12,0.064516,10,0.053763,1,0.005376,1,0.005376,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773,28388,MVDGNYSVASNVMVPMRDGVRLAVDLYRPDADGPVPVLLVRNPYDK...,8.0,10.1093/protein/gzq025,38.5,574,57,0.099303,4,0.006969,...,0.067944,36,0.062718,54,0.094077,13,0.022648,17,0.029617,309
774,28389,MVDGNYSVASNVMVPMRDGVRLAVDLYRPDADGPVPVLLVRNPYDK...,8.0,10.1093/protein/gzq025,37.7,574,57,0.099303,4,0.006969,...,0.067944,37,0.064460,54,0.094077,13,0.022648,17,0.029617,309
775,28840,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,7.8,NaN,3.8,154,16,0.103896,0,0.000000,...,0.038961,5,0.032468,8,0.051948,2,0.012987,3,0.019481,1
776,29131,MVSLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHT...,7.8,NaN,15.0,159,13,0.081761,2,0.012579,...,0.056604,6,0.037736,12,0.075472,5,0.031447,4,0.025157,1


,seq_id,protein_sequence,pH,data_source,tm,n_AA,AA_A__count,AA_A__fraction,AA_C__count,AA_C__fraction,...,AA_S__fraction,AA_T__count,AA_T__fraction,AA_V__count,AA_V__fraction,AA_W__count,AA_W__fraction,AA_Y__count,AA_Y__fraction,data_source_enum
0,1021,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFQNRESVLPT...,8.5,10.1016/j.jmb.2005.09.091,50.4,96,7,0.072917,2,0.020833,...,0.072917,13,0.135417,6,0.062500,0,0.000000,8,0.083333,6
1,1031,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFQNRESVLPT...,8.5,10.1016/j.jmb.2005.09.091,42.3,96,6,0.062500,2,0.020833,...,0.072917,13,0.135417,6,0.062500,0,0.000000,8,0.083333,6
2,1052,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFQNRESVLPT...,8.5,10.1016/j.jmb.2005.09.091,51.8,96,6,0.062500,2,0.020833,...,0.072917,13,0.135417,6,0.062500,0,0.000000,8,0.083333,6
3,1065,DVSGTVCLSALPPEATDTLNLIASDGPFPYSQDGVVFQNRESVLPT...,8.5,10.1016/j.jmb.2005.09.091,46.3,96,6,0.062500,2,0.020833,...,0.072917,13,0.135417,6,0.062500,0,0.000000,8,0.083333,6
4,3318,MAARLLRVASAALGDTAGRWRLLARPRAGAGGLRGSRGPGLGGGAV...,8.5,NaN,25.0,186,20,0.107527,5,0.026882,...,0.075269,12,0.064516,10,0.053763,1,0.005376,1,0.005376,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773,28388,MVDGNYSVASNVMVPMRDGVRLAVDLYRPDADGPVPVLLVRNPYDK...,8.0,10.1093/protein/gzq025,38.5,574,57,0.099303,4,0.006969,...,0.067944,36,0.062718,54,0.094077,13,0.022648,17,0.029617,309
774,28389,MVDGNYSVASNVMVPMRDGVRLAVDLYRPDADGPVPVLLVRNPYDK...,8.0,10.1093/protein/gzq025,37.7,574,57,0.099303,4,0.006969,...,0.067944,37,0.064460,54,0.094077,13,0.022648,17,0.029617,309
775,28840,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,7.8,NaN,3.8,154,16,0.103896,0,0.000000,...,0.038961,5,0.032468,8,0.051948,2,0.012987,3,0.019481,1
776,29131,MVSLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHT...,7.8,NaN,15.0,159,13,0.081761,2,0.012579,...,0.056604,6,0.037736,12,0.075472,5,0.031447,4,0.025157,1


In [37]:


for _aa_char in aa_chars_ordered:    
    # Check Correlations
    print(f"AA_{_aa_char} has a spearman correlation of : (" \
          f"{np.abs(train_8ph_df[f'AA_{_aa_char}__count'].corr(train_8ph_df['tm'], 'spearman')):.3f}, " \
          f"{np.abs(train_8ph_df[f'AA_{_aa_char}__fraction'].corr(train_8ph_df['tm'], 'spearman')):.3f}) (count, fraction)")

# Perhaps the most commonly configured hyperparameters are the following:
#    - n_estimators     : The number of trees in the ensemble, often increased until no further improvements are seen.
#    - max_depth        : The maximum depth of each tree, often values are between 1 and 10.
#    - eta              : The learning rate used to weight each model, often set to small values such as 0.3, 0.1, 0.01, or smaller.
#    - subsample        : The number of samples (rows) used in each tree, set to a value between 0 and 1, often 1.0 to use all samples.
#    - colsample_bytree : Number of features (columns) used in each tree, set to a value between 0 and 1, often 1.0 to use all features.
baseline_model = XGBRegressor(n_estimators=1000, learning_rate=0.1, max_depth=7, n_jobs = -1)

# training on features with '__' in the name, as well as n_AA, and pH. Targets are, of course, 'tm'
baseline_model.fit(X=train_df[[_c for _c in train_df.columns if "__" in _c]+["n_AA", "pH"]].to_numpy(), y=train_df["tm"].to_numpy(), verbose=True)

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=7)

# evaluate model
scores = cross_val_score(baseline_model, X=train_df[[_c for _c in train_df.columns if "__" in _c]+["n_AA", "pH"]].to_numpy(), scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

preds = baseline_model.predict(test_df[[_c for _c in test_df.columns if "__" in _c]+["n_AA", "pH"]].to_numpy())
ss_df["tm"] = preds
ss_df.to_csv("submission.csv", index=False)
ss_df

AA_A has a spearman correlation of : (0.557, 0.157) (count, fraction)
AA_C has a spearman correlation of : (0.392, 0.091) (count, fraction)
AA_D has a spearman correlation of : (0.337, 0.559) (count, fraction)
AA_E has a spearman correlation of : (0.347, 0.237) (count, fraction)
AA_F has a spearman correlation of : (0.448, 0.009) (count, fraction)
AA_G has a spearman correlation of : (0.551, 0.629) (count, fraction)
AA_H has a spearman correlation of : (0.505, 0.362) (count, fraction)
AA_I has a spearman correlation of : (0.496, 0.101) (count, fraction)
AA_K has a spearman correlation of : (0.412, 0.063) (count, fraction)
AA_L has a spearman correlation of : (0.469, 0.079) (count, fraction)
AA_M has a spearman correlation of : (0.485, 0.152) (count, fraction)
AA_N has a spearman correlation of : (0.451, 0.267) (count, fraction)
AA_P has a spearman correlation of : (0.476, 0.127) (count, fraction)
AA_Q has a spearman correlation of : (0.569, 0.400) (count, fraction)
AA_R has a spearman 

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning:


30 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 678, in _fit_and_score
    estimator.fit(X_train, **fit_params)
TypeError: fit() missing 1 required positional argument: 'y'




,seq_id,tm
0,31390,45.789520
1,31391,46.154991
2,31392,45.600948
3,31393,44.637577
4,31394,46.656300
...,...,...
2408,33798,45.028748
2409,33799,45.907814
2410,33800,46.047230
2411,33801,48.026924


# Starting to use Deepddg

The following file that we are importing contains the ddg values for every single point mutation in our wildtype proein. To generate this type of file for a given sequence, you can use the [DDG server](http://protein.org.cn/ddg.html).

In [50]:
# 1.1 – Define the path to the model output text file (I'm using mine but you can use the original)
DEEPDDG_PRED_TXT_PATH = "/content/drive/My Drive/Synbio/wildtype_structure_prediction_af2.deepddg.ddg.txt"

In [51]:
# 1.2 – Read the txt file into a pandas dataframe specifying a space as the delimiter
#       --> This introduces some weirdness with the number of columns... so we have to drop some and rename them
#       --> We also drop the #chain column as it contains no usable information (all rows are the same --> A)
#       --> We also rename the columns to be more user friendly (note ddg stands for ΔΔG)
deepddg_pred_df = pd.read_table(DEEPDDG_PRED_TXT_PATH, sep=" ").drop(columns=["#chain", "ddG", "is", "stable,", "is.1", "unstable)", "<0"])
deepddg_pred_df.columns = ["wildtype_aa", "residue_id", "mutant_aa", "ddg", "ddg_"]

In [52]:
# 1.3 – Coerce all the ddg values to be in the right column
deepddg_pred_df["ddg"] = deepddg_pred_df["ddg"].fillna(0.0)+deepddg_pred_df["ddg_"].fillna(0.0)
deepddg_pred_df = deepddg_pred_df.drop(columns=["ddg_"])

In [53]:
# 1.4 – Change edit location string name and change from 1-indexed to 0-indexed
deepddg_pred_df.loc[:,'location'] = deepddg_pred_df["residue_id"]-1
deepddg_pred_df = deepddg_pred_df.drop(columns=["residue_id"])

In [54]:
deepddg_pred_df

,wildtype_aa,mutant_aa,ddg,location
0,V,A,-0.119,0
1,V,R,-0.132,0
2,V,N,-0.155,0
3,V,D,-0.158,0
4,V,C,-0.162,0
...,...,...,...,...
4194,K,S,-1.523,220
4195,K,T,-2.530,220
4196,K,W,-1.325,220
4197,K,Y,-1.250,220


In [55]:
# 1.5 – Create a new column that contains the entire mutation as a string
#   --> This mutation string has the format   <wildtype_aa><location><mutant_aa>
deepddg_pred_df.loc[:,'mutant_string'] = deepddg_pred_df["wildtype_aa"]+deepddg_pred_df["location"].astype(str)+deepddg_pred_df["mutant_aa"]

In [56]:
# 1.6 – Display the newly created datafarme containing predictions (and describe float/int based columns)
display(deepddg_pred_df.describe().T)
display(deepddg_pred_df)

,count,mean,std,min,25%,50%,75%,max
ddg,4199.0,-1.641449,1.52908,-6.263,-2.589,-1.079,-0.453,3.153
location,4199.0,110.000000,63.80415,0.000,55.000,110.000,165.000,220.000


,wildtype_aa,mutant_aa,ddg,location,mutant_string
0,V,A,-0.119,0,V0A
1,V,R,-0.132,0,V0R
2,V,N,-0.155,0,V0N
3,V,D,-0.158,0,V0D
4,V,C,-0.162,0,V0C
...,...,...,...,...,...
4194,K,S,-1.523,220,K220S
4195,K,T,-2.530,220,K220T
4196,K,W,-1.325,220,K220W
4197,K,Y,-1.250,220,K220Y
